![](../docs/banner.png)

# Sắp xếp và biến đổi dữ liệu

Trong nhiều ứng dụng, dữ liệu có thể nằm rải rác trên nhiều tệp hoặc cơ sở dữ liệu, hoặc được sắp xếp dưới dạng không thuận tiện cho việc phân tích. Chương này tập trung vào các công cụ giúp kết hợp, nối và sắp xếp lại dữ liệu.

Đầu tiên, chúng ta sẽ làm quen với khái niệm về chỉ số phân cấp (hierarchical indexing) trong `pandas`, một công cụ được sử dụng rộng rãi trong sắp xếp và biến đổi dữ liệu. Sau đó, chúng ta sẽ đi sâu vào các thao tác xử lý dữ liệu cụ thể.

## Chỉ số phân cấp
<hr>

Xét ví dụ sau, với bảng dữ liệu theo dõi giá cổ phiếu theo ngày

In [1]:
import pandas as pd
import numpy as np

stock_prices = pd.DataFrame({
    'Ma CP': ['VPB', 'VPB', 'TCB', 'TCB', 'VNM', 'VNM'],
    'Ngay': ['2025-08-14','2025-08-15','2025-08-14','2025-08-15', '2025-08-14','2025-08-15'],
    'Price': [31.2, 32.5, 35.1, 34.0, 60.5, 61.3]
})

stock_prices

,Ma CP,Ngay,Price
0,VPB,2025-08-14,31.2
1,VPB,2025-08-15,32.5
2,TCB,2025-08-14,35.1
3,TCB,2025-08-15,34.0
4,VNM,2025-08-14,60.5
5,VNM,2025-08-15,61.3


Một số thao tác cơ bản cần thiết với dữ liệu này gồm:    
1. Lấy dữ liệu giá của 1 cổ phiếu nào đó, ví dụ `VPB`
2. Lấy giá các cổ phiếu của 1 ngày nào đó, ví dụ `2025-08-14`

In [2]:
vpb_prices = stock_prices[stock_prices['Ma CP'] == 'VPB']
vpb_prices

,Ma CP,Ngay,Price
0,VPB,2025-08-14,31.2
1,VPB,2025-08-15,32.5


In [3]:
prices_by_date = stock_prices[stock_prices['Ngay'] == '2025-08-14']
prices_by_date

,Ma CP,Ngay,Price
0,VPB,2025-08-14,31.2
2,TCB,2025-08-14,35.1
4,VNM,2025-08-14,60.5


→ Như vậy chúng ta đã sử dụng các kỹ thuật lọc (boolean indexing) trong `DataFrame`. Với Hierarchical indexing chúng ta có thể làm cho kết quả tương tự nhưng dễ dàng và nhanh hơn rất nhiều.

In [4]:
stock_prices = stock_prices.set_index(keys=['Ma CP', 'Ngay'])
stock_prices

Price
Ma CP Ngay             
VPB   2025-08-14   31.2
      2025-08-15   32.5
TCB   2025-08-14   35.1
      2025-08-15   34.0
VNM   2025-08-14   60.5
      2025-08-15   61.3

In [5]:
stock_prices.index

MultiIndex([('VPB', '2025-08-14'),
            ('VPB', '2025-08-15'),
            ('TCB', '2025-08-14'),
            ('TCB', '2025-08-15'),
            ('VNM', '2025-08-14'),
            ('VNM', '2025-08-15')],
           names=['Ma CP', 'Ngay'])

### Truy xuất dữ liệu linh hoạt

- Lấy toàn bộ giá của mã 'VPB'

In [6]:
stock_prices.loc['VPB']


,Price
Ngay,
2025-08-14,31.2
2025-08-15,32.5


- Lấy giá các mã cổ phiếu của 1 ngày 

In [7]:
stock_prices.xs('2025-08-14', level='Ngay')

,Price
Ma CP,
VPB,31.2
TCB,35.1
VNM,60.5


### Tốc độ truy xuất xuất dữ liệu khi sử dụng index

In [8]:
import pandas as pd
import numpy as np

n = 1_000_000
df = pd.DataFrame({
    'Ma CP': np.random.choice(['VPB', 'TCB', 'VNM'], size=n),
    'Ngay': np.random.choice(['2025-08-14','2025-08-15'], size=n),
    'Price': np.random.choice([31.2, 32.5, 35.1, 34.0, 60.5, 61.3], n)
})

%time df[df['Ma CP']=='VPB']

CPU times: user 144 ms, sys: 2.82 ms, total: 147 ms
Wall time: 233 ms


,Ma CP,Ngay,Price
1,VPB,2025-08-14,31.2
2,VPB,2025-08-15,61.3
4,VPB,2025-08-14,34.0
10,VPB,2025-08-14,61.3
18,VPB,2025-08-15,32.5
...,...,...,...
999993,VPB,2025-08-14,31.2
999995,VPB,2025-08-14,61.3
999996,VPB,2025-08-14,61.3
999997,VPB,2025-08-15,60.5


In [9]:
df_index = df.set_index(['Ma CP', 'Ngay'])

In [10]:
%time df_index.loc['VPB']

CPU times: user 38.2 ms, sys: 1.25 ms, total: 39.5 ms
Wall time: 82.2 ms


,Price
Ngay,
2025-08-14,31.2
2025-08-15,61.3
2025-08-14,34.0
2025-08-14,61.3
2025-08-15,32.5
...,...
2025-08-14,31.2
2025-08-14,61.3
2025-08-14,61.3


### Thay đổi thứ tự các chỉ số phân cấp

In [11]:
stock_prices.swaplevel(0, 1)

,,Price
Ngay,Ma CP,
2025-08-14,VPB,31.2
2025-08-15,VPB,32.5
2025-08-14,TCB,35.1
2025-08-15,TCB,34.0
2025-08-14,VNM,60.5
2025-08-15,VNM,61.3


- Cách khác để lấ giá các mã cổ phiếu của 1 ngày

In [12]:
stock_prices.swaplevel(0, 1).loc['2025-08-14']

,Price
Ma CP,
VPB,31.2
TCB,35.1
VNM,60.5


### Sắp xếp dữ liệu theo chỉ số phân cấp

- Trước khi sắp xếp

In [13]:
print(stock_prices)

                  Price
Ma CP Ngay             
VPB   2025-08-14   31.2
      2025-08-15   32.5
TCB   2025-08-14   35.1
      2025-08-15   34.0
VNM   2025-08-14   60.5
      2025-08-15   61.3


- Sắp xếp theo index Mã CP

In [14]:
stock_prices.sort_index(level=0)

Price
Ma CP Ngay             
TCB   2025-08-14   35.1
      2025-08-15   34.0
VNM   2025-08-14   60.5
      2025-08-15   61.3
VPB   2025-08-14   31.2
      2025-08-15   32.5

- Sắp xếp theo Ngày

In [15]:
stock_prices.sort_index(level='Ngay')

,,Price
Ma CP,Ngay,
TCB,2025-08-14,35.1
VNM,2025-08-14,60.5
VPB,2025-08-14,31.2
TCB,2025-08-15,34.0
VNM,2025-08-15,61.3
VPB,2025-08-15,32.5


### Thống kê tóm tắt theo cấp

In [16]:
stock_prices.groupby(level=0).describe()

Price                                                    
      count   mean       std   min     25%    50%     75%   max
Ma CP                                                          
TCB     2.0  34.55  0.777817  34.0  34.275  34.55  34.825  35.1
VNM     2.0  60.90  0.565685  60.5  60.700  60.90  61.100  61.3
VPB     2.0  31.85  0.919239  31.2  31.525  31.85  32.175  32.5

In [17]:
stock_prices.groupby(level='Ma CP').mean()

,Price
Ma CP,
TCB,34.55
VNM,60.90
VPB,31.85


### Tạo chỉ số phân cấp từ các cột

Trong nhiều trường hợp chúng ta muốn sử dụng một vài cột dữ liệu thành index hàng, hoặc nghược lại.
- Ví dụ: Bảng dữ liệu doanh thu quí theo vùng và kênh như sau

In [18]:
import pandas as pd
data = {
    'Region': ['North','North','North','North','South','South','South','South'],
    'Channel': ['Online','Online','Store','Store','Online','Online','Store','Store'],
    'Year': [2024,2024,2024,2024,2025,2025,2025,2025],
    'Quarter': ['Q1','Q2','Q1','Q2','Q1','Q2','Q1','Q2'],
    'Revenue': [100,130,120,140,90,95,110,115]
}

df = pd.DataFrame(data)
print(df)

  Region Channel  Year Quarter  Revenue
0  North  Online  2024      Q1      100
1  North  Online  2024      Q2      130
2  North   Store  2024      Q1      120
3  North   Store  2024      Q2      140
4  South  Online  2025      Q1       90
5  South  Online  2025      Q2       95
6  South   Store  2025      Q1      110
7  South   Store  2025      Q2      115


- Đánh chỉ số phân cấp theo một số cột để dễ dàng quan sát và truy xuất dữ liệu hơn

In [19]:
df.set_index(['Region', 'Channel', 'Year', 'Quarter'])

Revenue
Region Channel Year Quarter         
North  Online  2024 Q1           100
                    Q2           130
       Store   2024 Q1           120
                    Q2           140
South  Online  2025 Q1            90
                    Q2            95
       Store   2025 Q1           110
                    Q2           115

- Khi đánh chỉ số phân cấp, mặc định các cột được dùng đánh chỉ số bị loại khỏi dữ liệu, nếu muốn giữ lại các cột đó:

In [20]:
df2 = df.set_index(['Region', 'Channel', 'Year', 'Quarter'], drop=False)
print(df2)

                            Region Channel  Year Quarter  Revenue
Region Channel Year Quarter                                      
North  Online  2024 Q1       North  Online  2024      Q1      100
                    Q2       North  Online  2024      Q2      130
       Store   2024 Q1       North   Store  2024      Q1      120
                    Q2       North   Store  2024      Q2      140
South  Online  2025 Q1       South  Online  2025      Q1       90
                    Q2       South  Online  2025      Q2       95
       Store   2025 Q1       South   Store  2025      Q1      110
                    Q2       South   Store  2025      Q2      115


In [21]:
df1 = df.set_index(['Region', 'Channel', 'Year', 'Quarter'])
print(df1)
df1.reset_index()

                             Revenue
Region Channel Year Quarter         
North  Online  2024 Q1           100
                    Q2           130
       Store   2024 Q1           120
                    Q2           140
South  Online  2025 Q1            90
                    Q2            95
       Store   2025 Q1           110
                    Q2           115


,Region,Channel,Year,Quarter,Revenue
0,North,Online,2024,Q1,100
1,North,Online,2024,Q2,130
2,North,Store,2024,Q1,120
3,North,Store,2024,Q2,140
4,South,Online,2025,Q1,90
5,South,Online,2025,Q2,95
6,South,Store,2025,Q1,110
7,South,Store,2025,Q2,115


### Chỉ số phân cấp cho cột

Đánh thêm chỉ số phân cấp cho cột giúp việc truy xuất, xử lý dữ liệu được thuận tiện hơn.

In [22]:
df1 = df.set_index(['Region', 'Channel', 'Year', 'Quarter'])
print(df1)

                             Revenue
Region Channel Year Quarter         
North  Online  2024 Q1           100
                    Q2           130
       Store   2024 Q1           120
                    Q2           140
South  Online  2025 Q1            90
                    Q2            95
       Store   2025 Q1           110
                    Q2           115


Bước 2: Biến `Year` và `Quarter` thành multi index cho cột
- `unstack()` để xoay index hàng thành cột


In [23]:
df2 = df1.unstack(['Year', 'Quarter'])
print(df2)

               Revenue                     
Year              2024          2025       
Quarter             Q1     Q2     Q1     Q2
Region Channel                             
North  Online    100.0  130.0    NaN    NaN
       Store     120.0  140.0    NaN    NaN
South  Online      NaN    NaN   90.0   95.0
       Store       NaN    NaN  110.0  115.0


- Sử dụng `stack()` để ép cột xuống thành hàng

In [24]:
df3 = df2.stack(['Year', 'Quarter'])
print(df3)

                             Revenue
Region Channel Year Quarter         
North  Online  2024 Q1         100.0
                    Q2         130.0
       Store   2024 Q1         120.0
                    Q2         140.0
South  Online  2025 Q1          90.0
                    Q2          95.0
       Store   2025 Q1         110.0
                    Q2         115.0


/tmp/ipykernel_44852/4169368457.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df3 = df2.stack(['Year', 'Quarter'])


- Truy xuất dữ liệu với multi-index hàng và cột

In [25]:
df2['Revenue', 2025]

Quarter            Q1     Q2
Region Channel              
North  Online     NaN    NaN
       Store      NaN    NaN
South  Online    90.0   95.0
       Store    110.0  115.0

In [26]:
df2.loc[('North'), ('Revenue',2024)]


Quarter,Q1,Q2
Channel,,
Online,100.0,130.0
Store,120.0,140.0


In [27]:
df2.loc[(slice(None),'Online'), ('Revenue',2024)]


,Quarter,Q1,Q2
Region,Channel,,
North,Online,100.0,130.0
South,Online,NaN,NaN


## Kết hợp các tập dữ liệu
<hr>

Dữ liệu thường nằm rải rác từ nhiều nguồn dữ liệu khác nhau, khi đó chúng ta cần ghép nối dự liệu từ các nguồn này lại với nhau để phục vụ quá trình phân tích, xử lý dữ liệu. 

Thư viện **pandas** cung cấp một số phương pháp linh hoạt và hiệu quả để thực hiện việc này, bao gồm:

- **`pandas.merge`**  
  Phương pháp này cho phép kết nối các hàng giữa hai hoặc nhiều `DataFrame` dựa trên một hoặc nhiều khóa chung. Cách tiếp cận này tương tự với các thao tác *join* trong ngôn ngữ truy vấn SQL và thường được sử dụng trong các hệ quản trị cơ sở dữ liệu quan hệ.

- **`pandas.concat`**
  Được sử dụng để ghép nối các đối tượng dọc theo một trục xác định (theo hàng hoặc theo cột). Phương pháp này thích hợp trong các tình huống cần kết hợp dữ liệu có cùng cấu trúc.

- **`combine_first`** 
  Dùng để kết hợp hai đối tượng có cấu trúc tương tự, trong đó các giá trị bị thiếu (*missing values*) trong một đối tượng sẽ được điền bởi các giá trị tương ứng từ đối tượng còn lại. Đây là phương pháp hữu ích trong việc làm sạch và bổ sung dữ liệu.


### pandas.merge: Gộp và nối dữ liệu

#### Tại sao?

Xét ví dụ về dữ liệu khách hàng như sau:
- Bảng 1: Lưu thông tin khách hàng, gồm: mã khách hàng, tên khách hàng, giới tính, năm sinh
- Bảng 2: Lưu thông tin giao dịch của khách hàng, gồm: mã đơn hàng, mã khách hàng, giá trị đơn hàng

Chúng ta cần phân tích xem khách hàng nào mua nhiều, giá trị trung bình mỗi đơn của từng khách là bao nhiêu,...

In [28]:
import pandas as pd
customers = pd.DataFrame({ 
                          'code': ['001', '002', '003', '004'],
                          'name': ['Hoa', 'Hùng', 'Linh', 'An'],
                          'gender': ['Nữ', 'Nam', 'Nữ', 'Nữ'],
                          'birthYear': [1985, 1999, 2001, 2005],
                          })
print(customers)

  code  name gender  birthYear
0  001   Hoa     Nữ       1985
1  002  Hùng    Nam       1999
2  003  Linh     Nữ       2001
3  004    An     Nữ       2005


In [29]:
orders = pd.DataFrame({
    'order_number': [100, 101, 102, 103, 104],
    'customer_code': ['001', '002', '001', '003', '001'],
    'amount': [120_000, 245_000, 150_000, 340_000, 230_000]
})
print(orders)

   order_number customer_code  amount
0           100           001  120000
1           101           002  245000
2           102           001  150000
3           103           003  340000
4           104           001  230000


Làm sao biết khách hàng tên `Hoa` đã mua bao nhiêu đơn?, giá trị trung bình đơn hàng là bao nhiêu?

#### Kết hợp dữ liệu trong **pandas**

In [30]:
merged = pd.merge(orders, customers, left_on='customer_code', right_on='code', how='inner')
print(merged)

   order_number customer_code  amount code  name gender  birthYear
0           100           001  120000  001   Hoa     Nữ       1985
1           101           002  245000  002  Hùng    Nam       1999
2           102           001  150000  001   Hoa     Nữ       1985
3           103           003  340000  003  Linh     Nữ       2001
4           104           001  230000  001   Hoa     Nữ       1985


Không thấy khách hàng tên `An` trong bảng tổng hợp?

In [31]:
merged_outer = pd.merge(orders, customers, left_on='customer_code', right_on='code', how='outer')
print(merged_outer)

   order_number customer_code    amount code  name gender  birthYear
0         100.0           001  120000.0  001   Hoa     Nữ       1985
1         102.0           001  150000.0  001   Hoa     Nữ       1985
2         104.0           001  230000.0  001   Hoa     Nữ       1985
3         101.0           002  245000.0  002  Hùng    Nam       1999
4         103.0           003  340000.0  003  Linh     Nữ       2001
5           NaN           NaN       NaN  004    An     Nữ       2005


**Bảng các giá trị khác nhau với đối số `how`**

| Tùy chọn      | Hành vi                                                                 |
|---------------|-------------------------------------------------------------------------|
| `how="inner"` | Chỉ sử dụng các tổ hợp khóa được quan sát thấy trong cả hai bảng           |
| `how="left"`  | Sử dụng tất cả các tổ hợp khóa được tìm thấy trong bảng bên trái            |
| `how="right"` | Sử dụng tất cả các tổ hợp khóa được tìm thấy trong bảng bên phải           |
| `how="outer"` | Sử dụng tất cả các tổ hợp khóa được quan sát thấy trong cả hai bảng cùng nhau |

#### Phép gộp nhiều-nhiều (many-to-many)

- Thế nào là quan hệ nhiều-nhiều (many-to-many)?
- Xét ví dụ sau:
    - Một khách hàng mua nhiều sản phẩm
    - Một khách  hàng thích nhiều sản phẩm

In [97]:
purchases = pd.DataFrame({
    'customer': ['Alice','Alice','Bob','Charlie','Charlie'],
    'product_purchased': ['Laptop','Mouse','Tablet','Phone','Mouse']
})

print(purchases)

  customer product_purchased
0    Alice            Laptop
1    Alice             Mouse
2      Bob            Tablet
3  Charlie             Phone
4  Charlie             Mouse


In [99]:
favorites = pd.DataFrame({
    'customer': ['Alice','Bob','Alice','Charlie'],
    'favorite_product': ['Keyboard','Mouse','Monitor','Laptop']
})

print(favorites)

  customer favorite_product
0    Alice         Keyboard
1      Bob            Mouse
2    Alice          Monitor
3  Charlie           Laptop


Cần thấy quan hệ giữa sản phẩm đã mua và sản phẩm yêu thích:
- Phát hiện sản phẩm khách chưa mua 
- Dự đoán nhu cầu -> xây dựng chương trình khuyến mại

In [100]:
purchases_favorite = pd.merge(purchases, favorites, on='customer', how='inner')
print(purchases_favorite)

  customer product_purchased favorite_product
0    Alice            Laptop         Keyboard
1    Alice            Laptop          Monitor
2    Alice             Mouse         Keyboard
3    Alice             Mouse          Monitor
4      Bob            Tablet            Mouse
5  Charlie             Phone           Laptop
6  Charlie             Mouse           Laptop


Từ kết quả này cho thấy: `Alice` đã mua Laptop và Mouse rồi, mà bàn phím và màn hình đang trong danh sách yêu thích của bạn ấy, nên khả năng cao bạn ấy sẽ mua các sản phẩm này.

**Bảng các tham số của hàm `pandas.merge`**

| Đối số        | Mô tả                                                                                                                                                             |
|---------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `left`        | DataFrame được gộp ở phía bên trái.                                                                                                                                |
| `right`       | DataFrame được gộp ở phía bên phải.                                                                                                                                |
| `how`         | Loại join để áp dụng: một trong số `"inner"`, `"outer"`, `"left"`, hoặc `"right"`; mặc định là `"inner"`.                                                               |
| `on`          | Tên cột để join. Phải được tìm thấy trong cả hai đối tượng DataFrame. Nếu không được chỉ định và không có khóa join nào khác được cung cấp, sẽ sử dụng giao điểm của các tên cột trong `left` và `right` làm khóa join. |
| `left_on`     | Các cột trong DataFrame `left` để sử dụng làm khóa join. Có thể là một tên cột duy nhất hoặc một danh sách các tên cột.                                                     |
| `right_on`    | Tương tự như `left_on` cho DataFrame `right`.                                                                                                                       |
| `left_index`  | Sử dụng chỉ mục hàng trong `left` làm khóa join của nó (hoặc các khóa, nếu là `MultiIndex`).                                                                          |
| `right_index` | Tương tự như `left_index`.                                                                                                                                          |
| `sort`        | Sắp xếp dữ liệu đã gộp theo thứ tự từ điển bằng các khóa join; `False` theo mặc định.                                                                                |
| `suffixes`    | Tuple các giá trị chuỗi để nối vào tên cột trong trường hợp chồng chéo; mặc định là `("_x", "_y")` (ví dụ: nếu "data" có trong cả hai đối tượng DataFrame, sẽ xuất hiện dưới dạng "data_x" và "data_y" trong kết quả). |
| `copy`        | Nếu `False`, tránh sao chép dữ liệu vào cấu trúc dữ liệu kết quả trong một số trường hợp ngoại lệ; theo mặc định luôn sao chép.                                       |
| `validate`    | Xác minh xem phép gộp có thuộc loại được chỉ định hay không, cho dù là một-một, một-nhiều, hay nhiều-nhiều. Xem docstring để biết chi tiết đầy đủ về các tùy chọn.     |
| `indicator`   | Thêm một cột đặc biệt `_merge` chỉ ra nguồn của mỗi hàng; các giá trị sẽ là `"left_only"`, `"right_only"`, hoặc `"both"` dựa trên nguồn gốc của dữ liệu được join trong mỗi hàng. |

### Kết hợp dữ liệu dựa trên chỉ số
<hr>

Trong một số trường hợp, các khóa dùng để kết hợp dữ liệu DataFrame không phải là cột mà là chỉ số hàng. Trong trường hợp này chúng ta sẽ thực hiện kết hợp theo chỉ số, sử dụng các đối số `left_index=True` hoặc `right_index=True` (hoặc cả hai) để chỉ ra rằng chỉ số được sử dụng làm khóa:

In [108]:
customers = pd.DataFrame({
    'customer_code': ['C001','C002','C003'],
    'name': ['Alice','Bob','Charlie'],
    'city': ['Hanoi','Saigon','Danang']
}).set_index('customer_code')
print(customers)

                  name    city
customer_code                 
C001             Alice   Hanoi
C002               Bob  Saigon
C003           Charlie  Danang


In [111]:
orders = pd.DataFrame({
    'order_id': [101,102,103,104],
    'customer_code': ['C001','C002','C002','C003'],
    'amount': [250,150,300,200]
}).set_index('order_id')

print(orders)


         customer_code  amount
order_id                      
101               C001     250
102               C002     150
103               C002     300
104               C003     200


In [112]:
orders_customers = pd.merge(orders, customers, left_on='customer_code', right_index=True)
print(orders_customers)

         customer_code  amount     name    city
order_id                                       
101               C001     250    Alice   Hanoi
102               C002     150      Bob  Saigon
103               C002     300      Bob  Saigon
104               C003     200  Charlie  Danang


**Lưu ý**

Khi thực hiện kết hợp dữ liệu dựa vào chỉ số sẽ tăng tốc độ lên nhiều lần, vì sao? code thử?

### Nối dữ liệu theo một trục
<hr>

Dữ liệu thực tế có thể bị chia nhỏ, lưu trữ tách biệt; ví dụ:
- Thu thập thông tin khách hàng được thực hiện bởi các nhân viên khác nhau
- Dữ liệu bán hàng, khách hàng của các đại lý

Khi đó để có dữ liệu hoàn chỉnh, đầy đủ thì chúng ta cần nối dữ liệu lại với nhau theo dòng (axis = 0) hoặc theo cột (axis = 1)

#### Nối dữ liệu theo dòng
- Giải sử có dữ liệu bán hàng của mỗi đại lý
- Cần tổng hợp lại thành một dữ liệu bán hàng thống nhất của tất cả đại lý

In [114]:
import pandas as pd

# Đại lý Hà Nội
sales_hanoi = pd.DataFrame({
    'product': ['Laptop', 'Mouse'],
    'quantity': [2, 5],
    'unit_price': [1200, 20],
})
sales_hanoi['amount'] = sales_hanoi['quantity'] * sales_hanoi['unit_price']
sales_hanoi['branch'] = 'Hanoi'

# Đại lý Sài Gòn
sales_saigon = pd.DataFrame({
    'product': ['Laptop', 'Keyboard'],
    'quantity': [1, 3],
    'unit_price': [1250, 30],
})
sales_saigon['amount'] = sales_saigon['quantity'] * sales_saigon['unit_price']
sales_saigon['branch'] = 'Saigon'

# Đại lý Đà Nẵng
sales_danang = pd.DataFrame({
    'product': ['Monitor', 'Mouse'],
    'quantity': [2, 4],
    'unit_price': [200, 18],
})
sales_danang['amount'] = sales_danang['quantity'] * sales_danang['unit_price']
sales_danang['branch'] = 'Danang'


In [115]:
print(sales_hanoi)

  product  quantity  unit_price  amount branch
0  Laptop         2        1200    2400  Hanoi
1   Mouse         5          20     100  Hanoi


In [116]:
print(sales_saigon)

    product  quantity  unit_price  amount  branch
0    Laptop         1        1250    1250  Saigon
1  Keyboard         3          30      90  Saigon


In [117]:
print(sales_danang)

   product  quantity  unit_price  amount  branch
0  Monitor         2         200     400  Danang
1    Mouse         4          18      72  Danang


- **Bài toán:** Tìm xem sản phẩm nào bán chạy nhất trên toàn hệ thống, tỷ trọng doanh thu từng sản phẩm? 

**Bước 1:** Tổng hợp dữ liệu bán hàng trên các đại lý

In [119]:
all_sales = pd.concat([sales_hanoi, sales_saigon, sales_danang], axis=0)
print(all_sales)

    product  quantity  unit_price  amount  branch
0    Laptop         2        1200    2400   Hanoi
1     Mouse         5          20     100   Hanoi
0    Laptop         1        1250    1250  Saigon
1  Keyboard         3          30      90  Saigon
0   Monitor         2         200     400  Danang
1     Mouse         4          18      72  Danang


**Bước 2:** Áp dụng tính toán tổng hợp trên dữ liệu gộp

In [121]:
all_sales_indexed = all_sales.set_index(['branch', 'product'])
print(all_sales_indexed)

                 quantity  unit_price  amount
branch product                               
Hanoi  Laptop           2        1200    2400
       Mouse            5          20     100
Saigon Laptop           1        1250    1250
       Keyboard         3          30      90
Danang Monitor          2         200     400
       Mouse            4          18      72


In [123]:
total_qty_by_product = all_sales_indexed.groupby('product')['quantity'].sum()
print(total_qty_by_product)

product
Keyboard    3
Laptop      3
Monitor     2
Mouse       9
Name: quantity, dtype: int64


In [127]:
total_revenue = all_sales_indexed['amount'].sum()

# Tỷ trọng doanh số theo chi nhánh
share_by_branch = all_sales_indexed.groupby('branch')['amount'].sum()/total_revenue * 100
print(share_by_branch)

print('-'*30)
# Tỷ trọng doanh số theo sản phẩm
share_by_product = all_sales_indexed.groupby('product')['amount'].sum()/total_revenue * 100
print(share_by_product)

branch
Danang    10.946197
Hanoi     57.977737
Saigon    31.076067
Name: amount, dtype: float64
------------------------------
product
Keyboard     2.087199
Laptop      84.647495
Monitor      9.276438
Mouse        3.988868
Name: amount, dtype: float64


#### Nối dữ liệu theo cột
- Dùng khi nhiều bảng chứa các thông tin khác nhau về cùng đối tượng. 
- Ví dụ:
       - Bộ phận chăm sóc khách hàng có thông tin khách hàng: Tên, giới tính, tuổi
       - Bộ phận kinh doanh: tổng chi tiêu của khách hàng, số đơn, tần suất mua hàng 
       - Bộ phận tài chính: thông tin về điểm tín dụng, lịch sử nợ/trả
- Để có được thông tin đầy đủ về từng khách hàng thì cần nối các thông tin này lại.

In [134]:
customers = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Anna'],
    'gener': [1, 1, 0, 0],
    'age': [20, 30, 40, 24]
}, index=[1, 2, 3, 4])
print(customers)

      name  gener  age
1    Alice      1   20
2      Bob      1   30
3  Charlie      0   40
4     Anna      0   24


In [137]:
purchases = pd.DataFrame({
    'total_spent': [5000, 2000, 3000]
}, index=[3, 4, 2])

print(purchases)


   total_spent
3         5000
4         2000
2         3000


In [138]:
scores = pd.DataFrame({
    'credit_score': [650, 720, 700]
}, index=[2, 3, 1])
print(scores)

   credit_score
2           650
3           720
1           700


- Tổng hợp dữ liệu bằng nối cột

In [139]:
customer_info = pd.concat([customers, purchases, scores],  axis=1)
print(customer_info)

      name  gener  age  total_spent  credit_score
1    Alice      1   20          NaN         700.0
2      Bob      1   30       3000.0         650.0
3  Charlie      0   40       5000.0         720.0
4     Anna      0   24       2000.0           NaN


**Bảng các tham số của hàm `pandas.concat`**

| Đối số             | Mô tả                                                                                                                                                              |
|--------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `objs`             | Danh sách hoặc từ điển các đối tượng pandas cần nối; đây là đối số bắt buộc duy nhất.                                                                                 |
| `axis`             | Trục để nối dọc theo; mặc định là nối dọc theo hàng (`axis="index"`).                                                                                                |
| `join`             | Hoặc `"inner"` hoặc `"outer"` (`"outer"` theo mặc định); liệu có lấy giao điểm (inner) hay hợp (outer) các chỉ số dọc theo các trục khác.                                |
| `keys`             | Các giá trị để liên kết với các đối tượng đang được nối, tạo thành một chỉ số phân cấp dọc theo trục nối; có thể là một danh sách hoặc mảng các giá trị tùy ý, một mảng các tuple, hoặc một danh sách các mảng (nếu nhiều mảng cấp được truyền vào `levels`). || `levels`           | Các chỉ mục cụ thể để sử dụng làm cấp hoặc các cấp chỉ mục phân cấp nếu `keys` được truyền.                                                                            |
| `names`            | Tên cho các cấp phân cấp được tạo nếu `keys` và/hoặc `levels` được truyền.                                                                                             |
| `verify_integrity` | Kiểm tra trục mới trong đối tượng được nối xem có trùng lặp không và đưa ra một ngoại lệ nếu có; theo mặc định (`False`) cho phép trùng lặp.                             |
| `ignore_index`     | Không bảo toàn các chỉ số dọc theo trục nối, thay vào đó tạo ra một chỉ số `range(total_length)` mới.                                                                   |

## Tổ chức và Cấu trúc lại dữ liệu
<hr>

- Khi làm với dữ liệu, tổ chức và cấu trúc lại dữ liệu (reshaping) là một kỹ thuật quan trọng nhằm giúp cho việc phân tích và trực quan hóa dữ liệu được dễ dàng hơn. Mục đích của reshaping là để chuyển đổi biểu diễn hay hình dạng (shape) của dữ liệu mà không làm thay đổi bản chất của dữ liệu đó. Có 2 kiểu biểu diễn dữ liệu như sau:
    -  Dạng rộng (wide): 
        - Mỗi đối tượng là một dòng, mỗi thuộc tính là một cột.
        - Ví dụ, dữ liệu điểm sinh viên, mỗi sinh viên một dòng, các cột là thông tin về sinh viên và điểm các môn.
        - Thường gặp ở đâu? từ các hệ thống phần mềm quản lý, bảng excel, các bảng báo cáo kết quả
        - Dùng cho: báo cáo, nhập liệu, trình bày
    - Dạng dài (long): 
        - Mỗi quan sát là một dòng, và cột là giá trị của quan sát đó.
        - Ví dụ: giá cổ phiếu theo ngày
        - Thường gặp ở đâu? Dữ liệu liên quan đến chuỗi thời gian, ghi nhận các quan sát liên tục,...
        - Dùng cho: phân tích, học máy, thống kê, trực quan hóa

- Sau đây chúng ta sẽ tìm hiểu chi tiết hai kỹ thuật rất quan trọng này.

### Chuyển đổi từ dạng `wide` sang `long`

- Ví dụ: 
    - Dữ liệu: Chúng ta có dữ liệu số lượng đơn hàng của các cửa hàng ở các thành phố khác nhau.
    - Yêu cầu: Tính trung bình lượng theo tháng toàn hệ thống

In [1]:
import pandas as pd

data = {
    'City': ['Hà Nội', 'TP.HCM', 'Đà Nẵng'],
    'Thang 1': [500, 700, 300],
    'Thang 2': [600, 800, 350],
    'Thang 3': [550, 750, 400]
}
df = pd.DataFrame(data)

print(df)

      City  Thang 1  Thang 2  Thang 3
0   Hà Nội      500      600      550
1   TP.HCM      700      800      750
2  Đà Nẵng      300      350      400


- Tính trung bình lượng đơn hàng theo tháng toàn hệ thống:
    - Tại sao phải chuyển đổi: Mỗi tháng 1 cột nên khó để groupby theo Tháng
    - Cần chuyển đôi sang dạng `long` với cột **Tháng**

In [8]:
df_long = df.melt(id_vars=['City'], value_vars=['Thang 1', 'Thang 2', 'Thang 3'], var_name='Month', value_name='NBOrders')
print(df_long)

      City    Month  NBOrders
0   Hà Nội  Thang 1       500
1   TP.HCM  Thang 1       700
2  Đà Nẵng  Thang 1       300
3   Hà Nội  Thang 2       600
4   TP.HCM  Thang 2       800
5  Đà Nẵng  Thang 2       350
6   Hà Nội  Thang 3       550
7   TP.HCM  Thang 3       750
8  Đà Nẵng  Thang 3       400


In [11]:
df_long.groupby('Month')['NBOrders'].mean()

Month
Thang 1    500.000000
Thang 2    583.333333
Thang 3    566.666667
Name: NBOrders, dtype: float64

### Chuyển đổi từ dạng long sang wide

Ví dụ: theo dõi chấm công nhân viên hàng ngày
- Hàng ngày ghi nhận số giờ làm việc của từng nhân viên
- Gửi báo cáo cho quản lý sao cho dễ dàng theo dõi, biết được thời gian làm việc của từng nhân viên trong tuần, tháng

In [8]:
import pandas as pd

df_long = pd.DataFrame({
    "Employee": ["An", "Bình", "An", "Bình"],
    "Date": ["2025-08-01", "2025-08-01", "2025-08-02", "2025-08-02"],
    "Hours": [8, 6, 7, 8]
})

print(df_long)


  Employee        Date  Hours
0       An  2025-08-01      8
1     Bình  2025-08-01      6
2       An  2025-08-02      7
3     Bình  2025-08-02      8


- Báo cáo chấm công cần dễ nhìn hơn, có thể thấy ngay mỗi nhân viên làm bao nhiều giờ mỗi ngày
    - Chuyển từ dạng **long** sang dạng **wide** bằng cách dùng hàm **pivot**

In [13]:
df_wide = df_long.pivot(index="Employee", columns="Date", values="Hours")

df_wide['total_hours'] = df_wide.sum(axis=1)
print(df_wide)


Date      2025-08-01  2025-08-02  total_hours
Employee                                     
An                 8           7           15
Bình               6           8           14


### Reshaping lại dữ liệu khi có chỉ số phân cấp

Khi làm việc với DataFrame có chỉ số phân cấp, ta có thể reshaping dữ liệu một cách linh hoạt nhờ hai thao tác chính:

* stack: “nén” một cấp cột xuống thành chỉ số hàng. Thường dùng để chuyển dữ liệu từ dạng **wide** sang dạng **long**.
* unstack: “giải nén” một cấp chỉ số hàng thành cột. Thường dùng để chuyển dữ liệu từ dạng **long** sang dạng **wide**.

Nói cách khác, stack và unstack chính là thao tác hoán đổi vị trí cấp chỉ số giữa rows và columns trong MultiIndex.


* Ví dụ bảng điểm sinh viên

In [18]:
import pandas as pd

df = pd.DataFrame({
    "Name": ["An", "Bình"],
    ("Score", "Math"): [8, 6],
    ("Score", "Physics"): [7, 9]
}).set_index('Name')

df.columns = pd.MultiIndex.from_tuples(df.columns)
print("=== Wide (MultiIndex columns) ===")
print(df)

=== Wide (MultiIndex columns) ===
     Score        
      Math Physics
Name              
An       8       7
Bình     6       9


* Kiểm tra chỉ số phân cấp (MultiIndex)

In [30]:
print(df.columns)

MultiIndex([('Score',    'Math'),
            ('Score', 'Physics')],
           )


* Như vậy MultiIndex có 2 cấp: 
    - level 0: Score
    - level 1: Math, Physics
* Dùng **stack** để "nén" 1 chỉ số cột xuống thành chỉ số hàng (từ **wide** sang **long**):

In [38]:
df_stack = df.stack(level=1, future_stack=True)
print(df_stack)

              Score
Name               
An   Math         8
     Physics      7
Bình Math         6
     Physics      9


* Dùng **unstack** để chuyển từ chỉ số dòng thành chỉ số cột (**long** sang **wide**):

In [37]:
df_stack.unstack(level=1)

Score        
      Math Physics
Name              
An       8       7
Bình     6       9